### Import packages and data

In [17]:
print("Importing Packages...")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mutual_info_score, accuracy_score, roc_auc_score, roc_curve,ConfusionMatrixDisplay
from sklearn.feature_extraction import DictVectorizer
from IPython.display import display
import warnings
from sklearn.exceptions import ConvergenceWarning
import pickle

Importing Packages...


In [18]:
print("Reading Data...")
df= pd.read_csv('data.csv')

Reading Data...


In [19]:
C = 1
n_splits = 10
output_file = f"Churn_Prediction_model_C= {C}.bin"

### Data Cleaning

In [20]:
print("Cleaning Data...")
df.columns = df.columns.str.lower().str.replace(' ','_')

Cleaning Data...


In [21]:
df.totalcharges = pd.to_numeric(df.totalcharges,errors = 'coerce')

In [22]:
numericals = ['tenure', 'monthlycharges', 'totalcharges']
categorical = ['gender', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [23]:
for c in categorical:
    df[c] = df[c].str.lower().str.replace(' ','_')

In [24]:
df.totalcharges = df.totalcharges.fillna(0)
df.churn = (df.churn == 'Yes').astype(int)

In [36]:
categorical = ['gender','seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [37]:
for c in categorical:
    print(c)
    df_group = df.groupby(c).churn.agg(['count','mean'])
    df_group['churn rate'] = df_group['mean'] - df.churn.mean()
    df_group['risk ratio'] = df_group['mean'] / df.churn.mean()
    display(df_group)
    print()

gender


,count,mean,churn rate,risk ratio
gender,,,,
female,3488,0.269209,0.003839,1.014466
male,3555,0.261603,-0.003766,0.985807



seniorcitizen


,count,mean,churn rate,risk ratio
seniorcitizen,,,,
0,5901,0.236062,-0.029308,0.889557
1,1142,0.416813,0.151443,1.570686



partner


...

,count,mean,churn rate,risk ratio
contract,,,,
month-to-month,3875,0.427097,0.161727,1.609440
one_year,1473,0.112695,-0.152675,0.424672
two_year,1695,0.028319,-0.237051,0.106714



paperlessbilling


,count,mean,churn rate,risk ratio
paperlessbilling,,,,
no,2872,0.163301,-0.102069,0.615371
yes,4171,0.335651,0.070281,1.264842



paymentmethod


,count,mean,churn rate,risk ratio
paymentmethod,,,,
bank_transfer_(automatic),1544,0.167098,-0.098271,0.629681
credit_card_(automatic),1522,0.152431,-0.112939,0.574410
electronic_check,2365,0.452854,0.187484,1.706502
mailed_check,1612,0.191067,-0.074303,0.720003


In [38]:
def mutual_info(column):
    return mutual_info_score(column,df.churn)
    

In [39]:
m_i = df[categorical].apply(mutual_info)
m_i.sort_values(ascending = False)

contract            0.098453
onlinesecurity      0.064677
techsupport         0.063021
internetservice     0.055574
onlinebackup        0.046792
paymentmethod       0.044519
deviceprotection    0.043917
streamingmovies     0.032001
streamingtv         0.031908
paperlessbilling    0.019194
dependents          0.014467
partner             0.011454
seniorcitizen       0.010577
multiplelines       0.000801
phoneservice        0.000072
gender              0.000037
dtype: float64

In [40]:
df[numericals].corrwith(df.churn)

tenure           -0.352229
monthlycharges    0.193356
totalcharges     -0.198324
dtype: float64

### Split data into Train, Validation and Test sets

In [27]:
print("Splitting into train,valid and test sets")
df_train_full,df_test = train_test_split(df,test_size = 0.2, random_state = 120)
df_train,df_val = train_test_split(df_train_full,test_size = 0.25, random_state =120) # test_size is 0.25 to get 20% of the full data

Splitting into train,valid and test sets


In [28]:
y_train = df_train.churn
y_val = df_val.churn
y_test = df_test.churn
del df_train['churn']
del df_val['churn']
del df_test['churn']

### Regularization parameter tuning with KFold Cross validation

In [29]:
def train(df_train,y_train,C):
    x_train = df_train.to_dict(orient = 'records')
    dv = DictVectorizer(sparse = False)
    x_train = dv.fit_transform(x_train)
    model = LogisticRegression(random_state = 120,C=C)
    model.fit(x_train,y_train)
    return dv,model

In [30]:
def predict(dv,model,df_val):
    x_val = df_val.to_dict(orient = 'records')
    x_val = dv.transform(x_val)
    y_pred = model.predict_proba(x_val)[:,1]
    return y_pred

In [32]:
print(f"performing cross validation with C = {C} and k = {n_splits}")
warnings.filterwarnings("ignore", category=ConvergenceWarning)
scores = []
kfold = KFold(n_splits = n_splits,shuffle = True, random_state = 120)
df_full_train = df_train_full.copy()
fold = 0
for train_idx,val_idx in (kfold.split(df_full_train)):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
    y_train = df_train.churn.values
    y_val = df_val.churn.values
    del df_train['churn']
    del df_val['churn']
    dv,model = train(df_train,y_train,C)
    y_pred = predict(dv,model,df_val)
    auc = roc_auc_score(y_val,y_pred)
    scores.append(auc)
    print(f"auc on fold {fold} is {auc}")
    fold+=1
print(f'Reg_param = {C}: auc = {np.mean(scores) +- np.std(scores)}')

performing cross validation with C = 1 and k = 10
auc on fold 0 is 0.8553150269541778
auc on fold 1 is 0.8304925987347214
auc on fold 2 is 0.8503963243546577
auc on fold 3 is 0.8400734089270498
auc on fold 4 is 0.8257686900177273
auc on fold 5 is 0.8574319696120555
auc on fold 6 is 0.8590687072614783
auc on fold 7 is 0.8620446108866331
auc on fold 8 is 0.829062082542361
auc on fold 9 is 0.8374433106575964
Reg_param = 1: auc = 0.8316966856589645


### Training the Final model

In [33]:
y_full_train = df_train_full.churn.values
del df_train_full['churn']
dv,model = train(df_train_full,y_full_train,C=0.01)
y_pred = predict(dv,model,df_test)
auc = roc_auc_score(y_test,y_pred)
print("auc = ",auc)

auc =  0.838290858870127


In [35]:
with open(output_file,'wb') as f_out:
    pickle.dump((dv,model),f_out)
print(f"Model Saved to the path {output_file}")


Model Saved to the path Churn_Prediction_model_C= 1.bin
